## Import libraries for HTTP requests and loading environment variables

`requests` is required for HTTP requests

`dotenv` is used in this guide to load properties from .env file. You can use different approach if you prefer

`json` is purely optional library to provide formated json output for demo purposes. It's not required to work with json at all

In [ ]:
import requests
from dotenv import dotenv_values
import json

# We need this to avoid warnings about disabled certificate verification. 
# In production you need to avoid verify=False and handle SSL properly.
import warnings
warnings.filterwarnings('ignore')

## Load environment settings and credentials from .env file
Please create `.env` file containing necessary environment and creadentials variables. Store it next to this jupyter notebook or your python code (otherwise, you can provide correct path to .env file as argument for `dotenv_values()`).<br/>
Variables to define:<br/>
- `GDP_BASE_URL` - base URL for Guardium appliance GUI. Should include protocol, host, port. Avoid trailing slash. `protocol://host:port`
- `CLIENT_ID, CLIENT_SECRET` - output of `grdapi register_oauth_client` cli command you need to run ahead of time on Guardium appliance
- `GUI_USER, GUI_PASSWORD` - Guardium GUI login creadentials of user with access to features you would like to access/manage via REST API

In [ ]:
env_vars = dotenv_values()
gdp_base_url=env_vars.get('GDP_BASE_URL')
client_id=env_vars.get('CLIENT_ID')
client_secret=env_vars.get('CLIENT_SECRET')
gui_user=env_vars.get('GUI_USER')
gui_password=env_vars.get('GUI_PASSWORD')

## Get token to access Guardium REST APIs

In [ ]:
oauth_url = f"{gdp_base_url}/oauth/token"
oauth_params = {
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'password',
    'username': gui_user,
    'password': gui_password
}

oauth_response = requests.post(oauth_url, params=oauth_params, verify=False)
oauth_response_body = oauth_response.json()

print("Oauth reponse body: ", json.dumps(oauth_response_body, indent=4))
access_token = oauth_response_body['access_token']
print("Access token: ", access_token)

#Store access token for Guardium REST API calls (we save it here as HTTP header)
auth_headers_for_restapi = {
    'Authorization': f'Bearer {access_token}'
}

## Now you can explore REST API endpoints available to your user

In [ ]:
restapi_list_url = f"{gdp_base_url}/restAPI/restapi"
restapi_list_params = {
    'withParameters': 'false', # true/false (default)
    'apiNameLike': '' #you can omit this parameter if you want to get a complete list of available functions
}

restapi_list_response = requests.get(restapi_list_url, headers=auth_headers_for_restapi, params=restapi_list_params, verify=False)

print(json.dumps(restapi_list_response.json(), indent=4))

## Report retreival via REST API

In [ ]:
online_report_url = f"{gdp_base_url}/restAPI/online_report"
online_report_data = {
    "reportName": "S-TAP Status Monitor",
    "reportParameter": {
        "SHOW_ALIASES": "Default", 
        "REMOTE_SOURCE": "%"
    }
}

online_report_response = requests.post(online_report_url, headers=auth_headers_for_restapi, json=online_report_data, verify=False)
print(json.dumps(online_report_response.json(), indent=4))

## Quick search

This REST API is a wrapper for Solr queries, which are used on Investigation Dashboard. The command is similar to the search command, but has some improvements.

This API is available in Guardium v11.3 and later. https://www.ibm.com/docs/en/guardium/11.5?topic=reference-quick-search

In [ ]:
quick_search_url = f"{gdp_base_url}/restAPI/quick_search"
quick_search_data = {
    "category": "ACCESS", 
    "inputTZ": "UTC",
    "startTime": "20240301 00:00:01", 
    "endTime": "20240326 23:00:11", 
    "filters":"name=DB Type&value=MS SQL SERVER"
}
quick_search_response = requests.post(quick_search_url, headers=auth_headers_for_restapi, json=quick_search_data, verify=False)
print(json.dumps(quick_search_response.json(), indent=4))

### Field titles

As you may have noticed, field keys for quick search results are not labels. You you want to show result in some report or on your customer GUI, `fieldTitled` can provide you with decoding of field labels

In [ ]:
field_titles_url = f"{gdp_base_url}/restAPI/fieldsTitles"
field_titles_response = requests.get(field_titles_url, headers=auth_headers_for_restapi, verify=False)
print(json.dumps(field_titles_response.json(), indent=4))

### Example of how one can combine quick search results with field labels

In [ ]:
def replace_keys_with_labels(obj, label_dict):
    if isinstance(obj, dict):
        return {label_dict.get(k, k): replace_keys_with_labels(v, label_dict) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_keys_with_labels(item, label_dict) for item in obj]
    else:
        return obj

quick_search_response_with_labels = replace_keys_with_labels(quick_search_response.json(), field_titles_response.json()[0])
print(json.dumps(quick_search_response_with_labels[0]["items"], indent=4))

## Group management

### Create group

In [ ]:
group_url = f"{gdp_base_url}/restAPI/group"
create_group_data = {
    "appid": "PUBLIC",
    "type": "USERS",
    "desc": "Guargium VUG group"
}
create_group_response = requests.post(group_url, headers=auth_headers_for_restapi, json=create_group_data, verify=False)
print(json.dumps(create_group_response.json(), indent=4))

group_desc_param = {
    "desc": "Guargium VUG group"
}
get_group_response = requests.get(group_url, headers=auth_headers_for_restapi, params=group_desc_param, verify=False)
print(json.dumps(get_group_response.json(), indent=4))

### Add member to group

In [ ]:
add_group_member_url = f"{gdp_base_url}/restAPI/group_member"
add_group_member_data = {
    "desc": "Guargium VUG group",
    "member": "Friendly member"
}
add_group_member_response = requests.post(add_group_member_url, headers=auth_headers_for_restapi, json=add_group_member_data, verify=False)
print(json.dumps(add_group_member_response.json(), indent=4))

get_group_members_url = f"{gdp_base_url}/restAPI/group_members_by_group_desc"
get_group_members_response = requests.get(get_group_members_url, headers=auth_headers_for_restapi, params=group_desc_param, verify=False)
print(json.dumps(get_group_members_response.json(), indent=4))

### Delete group

In [ ]:
delete_group_response = requests.delete(group_url, headers=auth_headers_for_restapi, params=group_desc_param, verify=False)
print(json.dumps(add_group_member_response.json(), indent=4))

## Updating users

There can be multiple different use cases when Guardium users may need update from external system.
Let's assume a situation when there is Privilieged Identity Management solution that needs to rotate passwords for Guardium cli user accounts (cli, guardcli1...guardcli9).

### Prerequisites
For user related operations it's necessary to use GUI user with accessmgr role (we need new access token for this exercise)

### Update guardcli1 password

In [ ]:
update_user_url = f"{gdp_base_url}/restAPI/user"
update_user_data = {
    "userName": "guardcli1",
    "password": "validPwd@123",
    "confirmPassword": "validPwd@123"
}
update_user_response = requests.put(update_user_url, headers=auth_headers_for_restapi, json=update_user_data, verify=False)
print(json.dumps(update_user_response.json(), indent=4))